In [1]:
#!/usr/bin/env python
import argparse
import math
import time
from dataclasses import dataclass
from typing import List, Tuple

import matplotlib.pyplot as plt

from pyvrp import Model
from pyvrp.plotting import plot_coordinates, plot_solution
from pyvrp.stop import MaxRuntime


@dataclass
class VRPTWInstance:
    coords: List[Tuple[float, float]]      # index = customer id (0 = depot)
    demands: List[int]                     # delivery demand
    ready_times: List[int]                 # time window lower bounds
    due_times: List[int]                   # time window upper bounds
    service_times: List[int]               # service durations
    vehicle_capacity: int
    num_vehicles: int
    depot_index: int = 0


def read_homberger_vrptw(path: str) -> VRPTWInstance:
    """
    Reads a Homberger/Solomon VRPTW instance in the format:

        VEHICLE
        NUMBER     CAPACITY
          50          200

        CUSTOMER
        CUST NO.  XCOORD.    YCOORD.    DEMAND   READY TIME  DUE DATE   SERVICE TIME

            0      70         70          0          0       1351          0
            1      33         78         20        750        809         90
            ...

    Returns a VRPTWInstance.
    """
    with open(path, "r") as f:
        # strip whitespace and skip completely empty lines
        lines = [ln.strip() for ln in f if ln.strip()]

    # --- VEHICLE section ---
    try:
        veh_idx = lines.index("VEHICLE")
    except ValueError:
        raise ValueError("Could not find 'VEHICLE' section in instance file.")

    # Next non-empty line after "NUMBER   CAPACITY" contains the values
    # Example:
    # NUMBER     CAPACITY
    #   50          200
    number_cap_header_idx = veh_idx + 1
    while number_cap_header_idx < len(lines) and not lines[number_cap_header_idx].split():
        number_cap_header_idx += 1

    number_cap_values_idx = number_cap_header_idx + 1
    while number_cap_values_idx < len(lines) and not lines[number_cap_values_idx].split():
        number_cap_values_idx += 1

    num_cap_tokens = lines[number_cap_values_idx].split()
    num_vehicles = int(num_cap_tokens[0])
    vehicle_capacity = int(num_cap_tokens[1])

    # --- CUSTOMER section ---
    try:
        cust_idx = lines.index("CUSTOMER")
    except ValueError:
        raise ValueError("Could not find 'CUSTOMER' section in instance file.")

    # Skip the header lines until we hit the first numeric line
    row_idx = cust_idx + 1
    # Skip header like "CUST NO.  XCOORD.  ..."
    while row_idx < len(lines) and not lines[row_idx].split()[0].isdigit():
        row_idx += 1

    rows = []
    max_cust_id = -1

    for ln in lines[row_idx:]:
        if ln.upper().startswith("EOF"):
            break

        tokens = ln.split()
        # Expect: ID, X, Y, DEMAND, READY, DUE, SERVICE
        if len(tokens) < 7:
            continue

        cust_id = int(tokens[0])
        x = float(tokens[1])
        y = float(tokens[2])
        demand = int(tokens[3])
        ready = int(tokens[4])
        due = int(tokens[5])
        service = int(tokens[6])

        rows.append((cust_id, x, y, demand, ready, due, service))
        max_cust_id = max(max_cust_id, cust_id)

    if max_cust_id < 0:
        raise ValueError("No customer lines found in CUSTOMER section.")

    n_locs = max_cust_id + 1  # include customer 0 (the depot)

    coords = [(0.0, 0.0)] * n_locs
    demands = [0] * n_locs
    ready_times = [0] * n_locs
    due_times = [0] * n_locs
    service_times = [0] * n_locs

    for cid, x, y, dem, r, d, serv in rows:
        coords[cid] = (x, y)
        demands[cid] = dem
        ready_times[cid] = r
        due_times[cid] = d
        service_times[cid] = serv

    return VRPTWInstance(
        coords=coords,
        demands=demands,
        ready_times=ready_times,
        due_times=due_times,
        service_times=service_times,
        vehicle_capacity=vehicle_capacity,
        num_vehicles=num_vehicles,
        depot_index=0,
    )


def build_model(inst: VRPTWInstance) -> Model:
    """
    Builds a PyVRP Model for a VRPTW instance with capacity, time windows,
    and service times.

    - Depot gets its own time window.
    - Each client has (delivery, service_duration, tw_early, tw_late).
    - Distances and durations are set to rounded Euclidean distance. :contentReference[oaicite:1]{index=1}
    """
    m = Model()

    depot_idx = inst.depot_index
    depot_x, depot_y = inst.coords[depot_idx]
    depot_ready = inst.ready_times[depot_idx]
    depot_due = inst.due_times[depot_idx]

    # Add depot with its time window
    depot = m.add_depot(
        x=depot_x,
        y=depot_y,
        tw_early=depot_ready,
        tw_late=depot_due,
        name="depot",
    )

    # Add clients
    for i, (x, y) in enumerate(inst.coords):
        if i == depot_idx:
            continue

        m.add_client(
            x=x,
            y=y,
            delivery=inst.demands[i],
            service_duration=inst.service_times[i],
            tw_early=inst.ready_times[i],
            tw_late=inst.due_times[i],
            name=f"cust_{i}",
        )

    # Add a single vehicle type (homogeneous fleet) with capacity and depot
    # Time-window at depot acts as route start/end horizon. :contentReference[oaicite:2]{index=2}
    m.add_vehicle_type(
        num_available=inst.num_vehicles,
        capacity=inst.vehicle_capacity,
        start_depot=depot,
        end_depot=depot,
        tw_early=depot_ready,
        tw_late=depot_due,
        # Let shift_duration be large (default) so that depot TW + customer TW drive feasibility.
    )

    # Fully connect all locations with Euclidean distance as both distance and duration
    locations = m.locations  # depots + clients in model order :contentReference[oaicite:3]{index=3}
    for frm in locations:
        for to in locations:
            dx = frm.x - to.x
            dy = frm.y - to.y
            dist = int(round(math.hypot(dx, dy)))
            m.add_edge(frm, to, distance=dist, duration=dist)

    return m


def solve_instance(
    path: str,
    time_limit: float = 60.0,
    seed: int = 0,
    display: bool = False,
    verbose: bool = True,
):
    """
    High-level helper:
      1. Read VRPTW instance from `path`.
      2. Build a PyVRP Model.
      3. Solve it with a MaxRuntime stop criterion.

    Returns (result, data) where:
      - result is a pyvrp.Result
      - data is the underlying ProblemData
    """
    inst = read_homberger_vrptw(path)

    if verbose:
        print("=== Instance info ===")
        print(f"File           : {path}")
        print(f"Customers      : {len(inst.coords) - 1}")
        print(f"Vehicles       : {inst.num_vehicles}")
        print(f"Capacity       : {inst.vehicle_capacity}")
        print(f"Depot TW       : [{inst.ready_times[inst.depot_index]}, "
              f"{inst.due_times[inst.depot_index]}]")
        print()

    m = build_model(inst)
    data = m.data()

    if verbose:
        print("=== Solving with PyVRP (VRPTW) ===")
        print(f"Time limit     : {time_limit} s")
        print(f"Seed           : {seed}")
        print()

    t0 = time.perf_counter()
    result = m.solve(
        stop=MaxRuntime(time_limit),
        seed=seed,
        display=display,
    )
    t1 = time.perf_counter()
    solve_time = t1 - t0

    best = result.best

    if verbose:
        print(result.summary())
        print()
        print("=== Solution stats (best found) ===")
        print(f"Feasible         : {best.is_feasible()}")
        print(f"Total distance   : {best.distance()}")
        print(f"Total duration   : {best.duration()}")
        print(f"Num routes       : {best.num_routes()}")
        print(f"Time warp (TW viol.) : {best.time_warp()}")
        print(f"Total overtime   : {best.overtime()}")
        print(f"Solve time [s]   : {solve_time: .2f}")

    return result, data, inst


def plot_instance_and_solution(result, data):
    """
    Convenience helper: plot coordinates and best solution
    in a side-by-side matplotlib figure using PyVRP's plotting tools. :contentReference[oaicite:4]{index=4}
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    plot_coordinates(data, ax=axes[0])
    axes[0].set_title("VRPTW instance")

    plot_solution(result.best, data, ax=axes[1])
    axes[1].set_title("PyVRP VRPTW solution")

    plt.tight_layout()
    plt.show()

def solve_vrptw(
    instance_path: str,
    time_limit: float = 60.0,
    seed: int = 0,
    display: bool = False,
    plot: bool = True,
    verbose: bool = True,
):
    """
    Notebook-friendly wrapper to solve a Homberger/Solomon VRPTW instance.

    Parameters:
    - instance_path: path to instance file (e.g. "C1_2_1.txt")
    - time_limit: solver time limit in seconds
    - seed: RNG seed
    - display: show PyVRP textual progress if True
    - plot: show matplotlib plots of instance + best solution if True
    - verbose: print solver/info messages

    Returns:
    (result, data, inst)  -- same as solve_instance
    """
    result, data, inst = solve_instance(
        path=instance_path,
        time_limit=time_limit,
        seed=seed,
        display=display,
        verbose=verbose,
    )

    if plot:
        plot_instance_and_solution(result, data)

    return result, data, inst


In [ ]:
sol = solve_vrptw(
    instance_path="Vrp-Set-HG/C1_2_1.txt",
    time_limit=120.0,
    seed=42,
    display=True,
    plot=True,
    verbose=True,
)

=== Instance info ===
File           : Vrp-Set-HG/C1_2_1.txt
Customers      : 200
Vehicles       : 50
Capacity       : 200
Depot TW       : [0, 1351]

=== Solving with PyVRP (VRPTW) ===
Time limit     : 120.0 s
Seed           : 42

PyVRP v0.12.1

Solving an instance with:
    1 depot
    200 clients
    50 vehicles (1 vehicle type)

                  |       Feasible        |      Infeasible
    Iters    Time |   #      Avg     Best |   #      Avg     Best
H     395      5s |  49     2719     2690 |  56     2629     2527
      646     10s |  28     2746     2690 |  61     2592     2512
      899     15s |  27     2738     2690 |  27     2751     2661
     1162     20s |  54     2728     2690 |  40     2583     2511
     1413     25s |  63     2737     2690 |  45     2599     2537
     1665     30s |  36     2721     2690 |  50     2761     2669
     1954     35s |  43     2722     2690 |  38     2578     2516
     2208     40s |  64     2745     2690 |  46     2590     2512
     2447  